In [2]:
# Cell 1: Import libraries
import pandas as pd
import re

In [3]:
# Cell 2: Load the movies dataset
movies = pd.read_csv('../backend/data/raw/movies.csv')
print(f"Original dataset shape: {movies.shape}")
print(f"Original number of movies: {len(movies)}")

Original dataset shape: (9742, 3)
Original number of movies: 9742


In [4]:
# Cell 3: Check for missing values
print("\nMissing values:")
print(movies.isnull().sum())


Missing values:
movieId    0
title      0
genres     0
dtype: int64


In [5]:
# Cell 4: Check for movies with no genres
no_genre = movies[movies['genres'] == '(no genres listed)']
print(f"\nMovies with no genres: {len(no_genre)}")


Movies with no genres: 34


In [6]:
# Cell 5: Check for duplicate movieId or title
duplicates = movies[movies.duplicated(subset=['movieId'], keep=False) | movies.duplicated(subset=['title'], keep=False)]
print(f"\nDuplicate entries: {len(duplicates)}")


Duplicate entries: 10


In [7]:
# Cell 6: Extract year from title and check for unusual years
def extract_year(title):
    match = re.search(r'\((\d{4})\)$', title)
    return int(match.group(1)) if match else None

movies['year'] = movies['title'].apply(extract_year)
unusual_years = movies[(movies['year'] < 1900) | (movies['year'] > 2023)]
print(f"\nMovies with unusual years: {len(unusual_years)}")


Movies with unusual years: 0


In [8]:
# Cell 7: Check number of genres per movie
movies['genre_count'] = movies['genres'].str.count('\|') + 1
genre_stats = movies['genre_count'].describe()
print("\nGenre count statistics:")
print(genre_stats)


Genre count statistics:
count    9742.000000
mean        2.266886
std         1.123249
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max        10.000000
Name: genre_count, dtype: float64


In [9]:
# Cell 8: Suggest filtering criteria
print("\nSuggested filtering criteria:")
print("1. Remove movies with no genres")
print("2. Remove duplicate entries")
print("3. Remove movies with years before 1900 or after 2023")


Suggested filtering criteria:
1. Remove movies with no genres
2. Remove duplicate entries
3. Remove movies with years before 1900 or after 2023


In [10]:
# Cell 9: Apply initial filters to movies dataset
filtered_movies = movies[
    (movies['genres'] != '(no genres listed)') &
    (~movies.duplicated(subset=['movieId'], keep='first')) &
    (~movies.duplicated(subset=['title'], keep='first')) &
    (movies['year'].between(1900, 2023))
]

print(f"\nAfter initial filtering:")
print(f"Dataset shape: {filtered_movies.shape}")
print(f"Number of movies: {len(filtered_movies)}")
print(f"Removed {len(movies) - len(filtered_movies)} movies")


After initial filtering:
Dataset shape: (9688, 5)
Number of movies: 9688
Removed 54 movies


In [11]:
# Cell 10: Load and apply ratings filter
# Load the filtered ratings dataset
filtered_ratings = pd.read_csv('../backend/data/processed/filtered_ratings.csv')

# Get list of movies that have enough ratings
movies_with_ratings = filtered_ratings['movieId'].unique()

# Update filtered_movies to only include movies with sufficient ratings
filtered_movies = filtered_movies[filtered_movies['movieId'].isin(movies_with_ratings)]

print(f"\nAfter ratings filter:")
print(f"Final dataset shape: {filtered_movies.shape}")
print(f"Final number of movies: {len(filtered_movies)}")
print(f"Removed by ratings filter: {len(movies) - len(filtered_movies)} movies")

# Show detailed breakdown
print("\nDetailed breakdown:")
print(f"Original movies: {len(movies)}")
print(f"After initial filters: {len(filtered_movies)}")
print(f"Movies with sufficient ratings: {len(movies_with_ratings)}")
print(f"Final movies count: {len(filtered_movies)}")


After ratings filter:
Final dataset shape: (3649, 5)
Final number of movies: 3649
Removed by ratings filter: 6093 movies

Detailed breakdown:
Original movies: 9742
After initial filters: 3649
Movies with sufficient ratings: 3650
Final movies count: 3649


In [35]:
# Cell 11: Clean, standardize and save final filtered dataset
print(f"Original count before standardization: {len(filtered_movies)}")

# Standardization functions
def standardize_title(title):
    # Move articles from end to beginning
    if ", The (" in title:
        title = "The " + title.replace(", The (", " (")
    if ", A (" in title:
        title = "A " + title.replace(", A (", " (")
    if ", An (" in title:
        title = "An " + title.replace(", An (", " (")
    
    # Remove any double spaces
    title = ' '.join(title.split())
    return title

def standardize_genres(genres):
    # Sort genres alphabetically for consistency
    return '|'.join(sorted(genres.split('|')))

def standardize_year(year):
    # Convert to integer
    return int(float(year))

# Count titles before standardization
the_count = sum(1 for title in filtered_movies['title'] if ", The (" in title)
a_count = sum(1 for title in filtered_movies['title'] if ", A (" in title)
an_count = sum(1 for title in filtered_movies['title'] if ", An (" in title)

# Apply standardization
filtered_movies['title'] = filtered_movies['title'].apply(standardize_title)
filtered_movies['genres'] = filtered_movies['genres'].apply(standardize_genres)
filtered_movies['year'] = filtered_movies['year'].apply(standardize_year)

# Print standardization stats
print("\nStandardization summary:")
print("------------------------")
print(f"Titles with 'The' at end before standardization: {the_count}")
print(f"Titles with 'A' at end before standardization: {a_count}")
print(f"Titles with 'An' at end before standardization: {an_count}")
print(f"Year range: {filtered_movies['year'].min()} to {filtered_movies['year'].max()}")
print(f"Number of unique genres: {len(filtered_movies['genres'].str.split('|').explode().unique())}")

# Save standardized dataset
filtered_movies.to_csv('../backend/data/processed/filtered_movies.csv', 
                      index=False,
                      quoting=1,
                      encoding='utf-8')

print("\nStandardized dataset saved to '../backend/data/processed/filtered_movies.csv'")

Original count before standardization: 3649

Standardization summary:
------------------------
Titles with 'The' at end before standardization: 689
Titles with 'A' at end before standardization: 51
Titles with 'An' at end before standardization: 10
Year range: 1902 to 2018
Number of unique genres: 19

Standardized dataset saved to '../backend/data/processed/filtered_movies.csv'


In [37]:
# Cell 12: Remove IMAX from genres and update genre counts
print("Movies containing IMAX before cleanup:")
imax_movies = filtered_movies[filtered_movies['genres'].str.contains('IMAX')]
for _, movie in imax_movies.iterrows():
    print(f"- {movie['title']}")
    print(f"  Old genres: {movie['genres']}")
    print(f"  Old genre count: {movie['genre_count']}")
    print()

# Remove IMAX from genres and update counts
def remove_imax(genres):
    genres_list = genres.split('|')
    genres_list = [g for g in genres_list if g != 'IMAX']
    return '|'.join(sorted(genres_list))

# Apply changes
filtered_movies['genres'] = filtered_movies['genres'].apply(remove_imax)
filtered_movies['genre_count'] = filtered_movies['genres'].str.count('\\|') + 1

# Show the changes
print("\nAfter IMAX removal:")
for _, movie in imax_movies.iterrows():
    updated = filtered_movies[filtered_movies['movie_id'] == movie['movie_id']].iloc[0]
    print(f"- {updated['title']}")
    print(f"  New genres: {updated['genres']}")
    print(f"  New genre count: {updated['genre_count']}")
    print()

# Save updated dataset
filtered_movies.to_csv('../backend/data/processed/filtered_movies.csv', 
                      index=False,
                      quoting=1,
                      encoding='utf-8')

print("\nUpdated dataset saved to '../backend/data/processed/filtered_movies.csv'")

Movies containing IMAX before cleanup:
- Apollo 13 (1995)
  Old genres: Adventure|Drama|IMAX
  Old genre count: 3

- The Lion King (1994)
  Old genres: Adventure|Animation|Children|Drama|IMAX|Musical
  Old genre count: 6

- Beauty and the Beast (1991)
  Old genres: Animation|Children|Fantasy|IMAX|Musical|Romance
  Old genre count: 6

- Everest (1998)
  Old genres: Documentary|IMAX
  Old genre count: 2

- Fantasia 2000 (1999)
  Old genres: Animation|Children|IMAX|Musical
  Old genre count: 4

- Star Wars: Episode II - Attack of the Clones (2002)
  Old genres: Action|Adventure|IMAX|Sci-Fi
  Old genre count: 4

- Treasure Planet (2002)
  Old genres: Adventure|Animation|Children|IMAX|Sci-Fi
  Old genre count: 5

- The Matrix Reloaded (2003)
  Old genres: Action|Adventure|IMAX|Sci-Fi|Thriller
  Old genre count: 5

- The Matrix Revolutions (2003)
  Old genres: Action|Adventure|IMAX|Sci-Fi|Thriller
  Old genre count: 5

- Harry Potter and the Prisoner of Azkaban (2004)
  Old genres: Adventure